# Deploy ADK to Agent Engine


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FDeploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-06-03 |
| google-adk | 1.1.1 |
| vertexai | 1.93.1 |

## Environment

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:
import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install google-adk==1.1.1

In [5]:
from google import adk

In [6]:
adk.__version__

'1.1.1'

In [7]:
vertexai.__version__

'1.95.0'

## AE 101

In [8]:
from vertexai import agent_engines

In [9]:
def delete(resource_id: str) -> None:
    remote_agent = agent_engines.get(resource_id)
    remote_agent.delete(force=True)
    print(f"Deleted remote agent: {resource_id}")

def list_agents() -> None:
    remote_agents = agent_engines.list()
    TEMPLATE = '''
{agent.name} ("{agent.display_name}")
- Create time: {agent.create_time}
- Update time: {agent.update_time}
'''
    remote_agents_string = '\n'.join(TEMPLATE.format(agent=agent) for agent in remote_agents)
    print(f"All remote agents:\n{remote_agents_string}")

In [10]:
list_agents()

All remote agents:

5147799092037943296 ("")
- Create time: 2025-06-02 14:14:28.600149+00:00
- Update time: 2025-06-02 14:18:55.955685+00:00


2164164338904989696 ("")
- Create time: 2025-06-02 11:57:47.050939+00:00
- Update time: 2025-06-02 12:01:18.059991+00:00


6087608854902734848 ("DICE_AGENT")
- Create time: 2025-04-25 08:19:20.817490+00:00
- Update time: 2025-04-25 08:22:05.936293+00:00


7916070303615156224 ("llm_auditor")
- Create time: 2025-04-25 07:13:25.339138+00:00
- Update time: 2025-04-25 07:18:12.898631+00:00


5307078744484085760 ("llm_auditor")
- Create time: 2025-04-09 08:22:36.179879+00:00
- Update time: 2025-04-09 08:25:42.734584+00:00


8402318325882814464 ("")
- Create time: 2025-03-28 08:42:23.121480+00:00
- Update time: 2025-03-28 08:46:29.629436+00:00


66410502317670400 ("my-awesome-agent")
- Create time: 2025-03-07 01:22:05.930759+00:00
- Update time: 2025-03-07 01:33:02.201959+00:00



## ADK Agent

In [11]:
from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

MODEL = "gemini-2.0-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=simple_search_agent, app_name=APP_NAME, session_service=session_service)

# Agent Interaction
def call_agent(runner, session, query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  events = runner.run(user_id=session.user_id, session_id=session.id, new_message=content)
  return events

events = call_agent(runner, session, "what is next Google IO date?")
list(events)

[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Google I/O 2025 will be held on May 20-21, 2025. The event will be livestreamed on YouTube from the Shoreline Amphitheater in Mountain View.\n')], role='model'), grounding_metadata=GroundingMetadata(grounding_chunks=[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='blog.google', title='blog.google', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXFnhc8Ovsa-pom3j0C-rGuM2BnYhs5te8eZ8WkgXX-PKC1s4H3Xfz_rHeFPyYsUunDaPP1gYNwJGr8ppKGgQqkPgTDf_WxjIKD3DtOfAk9golvASWLBqalvBNTtrYn-vERtM18i7WuQ2sAAEvMbvDNL')), GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain='indianexpress.com', title='indianexpress.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEkqxpJoywEaZ-OuII5oAqvjTN5yQsHUjxLbRDaM5Pl4IFgMH0N3Mm6hw

In [ ]:
#

## Agent Engine Local

In [12]:
from vertexai.preview import reasoning_engines

app = reasoning_engines.AdkApp(
    agent=simple_search_agent,
    enable_tracing=True,
)

In [14]:
session = app.create_session(user_id="u_123")
session

Session(id='5178732b-7fbe-4ec9-a817-86b4d5346043', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748930925.919127)

In [15]:
app.list_sessions(user_id="u_123")

ListSessionsResponse(sessions=[Session(id='674a43f0-55b8-431f-bc02-879a3b889d44', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748930918.709493), Session(id='5178732b-7fbe-4ec9-a817-86b4d5346043', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748930925.919127)])

In [16]:
my_session = app.get_session(user_id="u_123", session_id=session.id)

In [17]:
my_session

Session(id='5178732b-7fbe-4ec9-a817-86b4d5346043', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1748930925.919127)

In [18]:
for event in app.stream_query(
    user_id="u_123",
    session_id=my_session.id,
    message="whats the weather in new york",
):
  print(event)

{'content': {'parts': [{'text': 'The weather in New York is currently clear with a temperature of 59°F (15°C), but it feels like 59°F (15°C) with 61% humidity. The chance of rain is around 0%.\n\nThe forecast for the next few days is:\n\n*   **Tuesday:** Sunny with a high near 77°F.\n*   **Wednesday:** Sunny with a high near 80°F.\n*   **Thursday:** Partly cloudy with a high near 89°F. There is a 20% chance of rain during the day and 15% at night.\n*   **Friday:** Rainy with a high near 84°F and a 55% chance of rain during the day and 50% at night.'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'google.com', 'title': 'Weather information for locality: New York, administrative_area: NY', 'uri': 'https://www.google.com/search?q=weather+in+New York,+NY'}}, {'web': {'domain': 'weather.gov', 'title': 'weather.gov', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXHL6xS-CiE9mnqgh2WDwObz_klHS34GRGaed9GU8zCmq5fhxr7gRpvyHVGaz0LV5

## Agent Engine Remote

In [19]:
from vertexai import agent_engines

In [20]:
remote_app = agent_engines.create(
    agent_engine=simple_search_agent,
    requirements=[
        "google-cloud-aiplatform[adk,agent_engines]"
    ]
)

INFO:vertexai.agent_engines:Deploying google.adk.agents.Agent as an application.
INFO:vertexai.agent_engines:Identified the following requirements: {'google-cloud-aiplatform': '1.95.0', 'pydantic': '2.11.5', 'cloudpickle': '3.1.1'}
INFO:vertexai.agent_engines:The following requirements are appended: {'pydantic==2.11.5', 'cloudpickle==3.1.1'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[adk,agent_engines]', 'pydantic==2.11.5', 'cloudpickle==3.1.1']
INFO:vertexai.agent_engines:Using bucket agent_engine_deploy_staging
INFO:vertexai.agent_engines:Wrote to gs://agent_engine_deploy_staging/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://agent_engine_deploy_staging/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://agent_engine_deploy_staging/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vert

In [21]:
remote_app.resource_name

'projects/160214671203/locations/us-central1/reasoningEngines/8348169577238102016'

In [22]:
# Get the previously created AE App
# remote_app = agent_engines.get('projects/160214671203/locations/us-central1/reasoningEngines/2164164338904989696')

In [23]:
remote_session = remote_app.create_session(user_id="u_456")
remote_session

{'appName': '8348169577238102016',
 'events': [],
 'state': {},
 'id': '6879789940285112320',
 'lastUpdateTime': 1748931431.570455,
 'userId': 'u_456'}

In [24]:
remote_app.list_sessions(user_id="u_456")

{'sessions': [{'appName': '8348169577238102016',
   'events': [],
   'state': {},
   'id': '6879789940285112320',
   'lastUpdateTime': 1748931431.570455,
   'userId': 'u_456'}]}

In [25]:
remote_app

resource name: projects/160214671203/locations/us-central1/reasoningEngines/8348169577238102016

In [26]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="whats the weather in Hong Kong",
):
    print(event)

{'content': {'parts': [{'text': 'The weather in Hong Kong on Tuesday, June 3, 2025, is light rain with a 20% chance of precipitation. The temperature is 82°F (28°C), but it feels like 91°F (33°C) due to 81% humidity.\n\nThe forecast for the next 11 days is:\n\n*   **Wednesday, June 4:** Light rain, with temperatures between 77°F (25°C) and 82°F (28°C).\n*   **Thursday, June 5:** Light rain, with temperatures between 79°F (26°C) and 83°F (28°C).\n*   **Friday, June 6:** Light rain, with temperatures between 80°F (27°C) and 84°F (29°C).\n*   **Saturday, June 7:** Partly cloudy during the day and light rain at night, with temperatures between 80°F (27°C) and 87°F (31°C).\n*   **Sunday, June 8:** Scattered showers during the day and light rain at night, with temperatures between 81°F (27°C) and 88°F (31°C).\n*   **Monday, June 9:** Light rain during the day and partly cloudy at night, with temperatures between 81°F (27°C) and 88°F (31°C).\n*   **Tuesday, June 10:** Partly cloudy during the

In [27]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="Remember my home location is Hong Kong.",
):
    print(event)

{'content': {'parts': [{'text': 'Okay, I will remember that your home location is Hong Kong. How can I help you today?\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 21, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 21}], 'prompt_token_count': 593, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 593}], 'total_token_count': 614, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-93c00e98-2d76-425f-b480-1a644ad4963e', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 't0Wj6Xez', 'timestamp': 1748931445.292232}


In [28]:
new_remote_app = agent_engines.get(remote_app.resource_name)

In [29]:
remote_session2 = remote_app.create_session(user_id="u_456")
remote_session2

{'appName': '8348169577238102016',
 'events': [],
 'state': {},
 'id': '9170996250709852160',
 'userId': 'u_456',
 'lastUpdateTime': 1748931452.629971}

In [30]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session["id"],
    message="what is my home location?",
):
    print(event)

{'content': {'parts': [{'text': 'Your home location is Hong Kong.\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 8}], 'prompt_token_count': 620, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 620}], 'total_token_count': 628, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-255505b6-1168-4fe6-9c8c-b9e29eaf77c4', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '3ikJYFPW', 'timestamp': 1748931456.751014}


In [31]:
for event in remote_app.stream_query(
    user_id="u_456",
    session_id=remote_session2["id"],
    message="what is my home location?",
):
    print(event)

{'content': {'parts': [{'text': 'I do not have access to personal information such as your home location. I am a language model; my purpose is to provide information based on publicly available data and complete the tasks I am given. I cannot ask you for your Personally Identifiable Information (PII).\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 53, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 53}], 'prompt_token_count': 53, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 53}], 'total_token_count': 106, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-38a5dc6a-888f-4e53-a9f8-701243840cd7', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'bt6zbyOA', 'timestamp': 1748931459.724485}


In [32]:
def call_remote_agent_engine_app(app, user_id, session_id, query):
  response_text = ''
  for event in app.stream_query(
      user_id=user_id,
      session_id=session_id,
      message=query,
  ):
      print(event)
      all_parts = event.get('content', {}).get('parts', [])
      for part in all_parts:
          response_text += (part.get('text', ''))
  return response_text

In [33]:
call_remote_agent_engine_app(remote_app, "u_456", remote_session2["id"], "what is my home location?")

{'content': {'parts': [{'text': "As a language model, I have no memory of past conversations and I don't know anything about your personal details. Therefore, I am unable to determine your home location.\n"}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 36, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 36}], 'prompt_token_count': 112, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 112}], 'total_token_count': 148, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-0f11c3b1-af0a-45f4-b0b5-1b57ef472791', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'ZYAEc1fQ', 'timestamp': 1748931470.198124}


"As a language model, I have no memory of past conversations and I don't know anything about your personal details. Therefore, I am unable to determine your home location.\n"

In [34]:
call_remote_agent_engine_app(remote_app, "u_456", remote_session["id"], "what is my home location?")

{'content': {'parts': [{'text': 'Your home location is Hong Kong.\n'}], 'role': 'model'}, 'grounding_metadata': {}, 'usage_metadata': {'candidates_token_count': 8, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 8}], 'prompt_token_count': 634, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 634}], 'total_token_count': 642, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-33ffcda7-40c9-448f-bd46-0f3a202ddb3d', 'author': 'simple_search_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'mFkNmuBP', 'timestamp': 1748931473.378676}


'Your home location is Hong Kong.\n'

## Mesop

In [35]:
!pip install mesop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [36]:
import mesop as me
import mesop.labs as mel

me.colab_run()


Running server on: http://localhost:32123
 * Serving Flask app 'mesop.server.server'
 * Debug mode: off


In [37]:
@me.page(path="/hello_world")
def hello_world():
  me.text("Hello World!")

In [38]:
me.colab_show(path="/hello_world", height=100)

<IPython.core.display.Javascript object>

In [39]:
@me.page(path="/chat")
def chat():
  mel.chat(transform)

def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
  return "Hello " + prompt

In [40]:
me.colab_show(path="/chat")

<IPython.core.display.Javascript object>

In [41]:
USER_ID = 'mesop_user_123'
SESSION = remote_app.create_session(user_id=USER_ID)

@me.page(path="/chat_ae")
def chat():
  mel.chat(transform)

def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
  # No need to use history, as AE already manages the session.
  return call_remote_agent_engine_app(remote_app, USER_ID, SESSION['id'], prompt)

In [42]:
me.colab_show(path="/chat_ae")

<IPython.core.display.Javascript object>

### Screenshots

The dynamic web UI may not present on the export notebook. Below are the screenshots.

In [43]:
from IPython.display import Image, display

In [44]:
display(Image(url='https://github.com/hupili/google-adk-101/raw/main/images/ss-ae-mesop-1.png'))

In [45]:
display(Image(url='https://github.com/hupili/google-adk-101/raw/main/images/ss-ae-mesop-2.png'))